# TP5

### Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.

### Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento.

### Comparar con alguno de los métodos vistos en la practica basados enmezcla de gaussianas
##### El detector de fondo Naive es 10 veces mas lento que el MOG2, por razones de calculo, aparte de tener que calcular la mediana de un nro N de frames.

In [33]:
import numpy as np
import cv2 as cv


In [34]:
# Función de detector de fondo naive
def Naive(N,frame,data):
    e1 = cv.getTickCount()  #empiezo a contar
    data.append(frame)

    if (len(data) > N):
        naive = np.array(data)
        naive = np.median(naive,axis=0)
        naive = naive.astype(np.uint8)
        cv.imshow('fondo Naive',naive)
        # data.clear()
        #En vez de borrar todo el array solo borramos el primer
        #frame del "array" para calcular la mediana y no perder mas frames
        data.pop(0)
        diff = cv.absdiff(frame_gray,naive)
        ret, thresh = cv.threshold(diff,20,255,cv.THRESH_BINARY)
        e2 = cv.getTickCount()  #Termino de contar
        time = (e2 - e1)/ cv.getTickFrequency()
        print('Tiempo de calculo Naive',time)
        #cargar el array demora 20mseg
        #pero una vez cargado procesar el dato alrededor de 15mseg
        #10 veces mas que el MOG2
    else:
        #Se pierden los primeros N frames antes de calcular la mediana
        thresh = frame

    return thresh

In [35]:
metodo = 'MOG2'
# metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [36]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)



In [37]:
# Corremos la sustraccion
#------------------------
ret, frame = capture.read()
averageValue1 = np.float32(frame)
data=[]

fgMask = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
# ****************
# N=cantidad de frames para estimaciòn
N = 20
# *****************

while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break

    # Aplicamos la sustracción al frame leído "media"

    frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    # N=cantidad de frames para estimaciòn
    # frame_gray = frame para crear el array
    # data = Array para calcular estimaciòn
    fgMask=Naive(N,frame_gray,data)
    #Mog2
    e1_mog = cv.getTickCount()  #empiezo a contar
    mog2=backSub.apply(frame)
    e2_mog = cv.getTickCount()  #empiezo a contar
    print('Tiempo de calculo MOG2',(e2_mog - e1_mog)/ cv.getTickFrequency())
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    # Escribimos sobre la imagen el número de frame naive procesado
    cv.rectangle(fgMask, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fgMask, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    cv.imshow('FG MOG2', mog2)
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

Tiempo de calculo MOG2 0.029791263
Tiempo de calculo MOG2 0.015354353
Tiempo de calculo MOG2 0.014786041
Tiempo de calculo MOG2 0.012268369
Tiempo de calculo MOG2 0.017433538
Tiempo de calculo MOG2 0.010947723
Tiempo de calculo MOG2 0.014526308
Tiempo de calculo MOG2 0.015909018
Tiempo de calculo MOG2 0.01677342
Tiempo de calculo MOG2 0.012260862
Tiempo de calculo MOG2 0.019170486
Tiempo de calculo MOG2 0.011689389
Tiempo de calculo MOG2 0.014879728
Tiempo de calculo MOG2 0.017474623
Tiempo de calculo MOG2 0.012589056
Tiempo de calculo MOG2 0.017765768
Tiempo de calculo MOG2 0.014071278
Tiempo de calculo MOG2 0.01628661
Tiempo de calculo MOG2 0.016086018
Tiempo de calculo MOG2 0.0128941
Tiempo de calculo Naive 0.218287306
Tiempo de calculo MOG2 0.012180402
Tiempo de calculo Naive 0.250980415
Tiempo de calculo MOG2 0.013189937
Tiempo de calculo Naive 0.199846679
Tiempo de calculo MOG2 0.018826449
Tiempo de calculo Naive 0.164467195
Tiempo de calculo MOG2 0.012211348
Tiempo de calculo Na